In [1]:
import pandas as pd
import numpy as np
import tqdm
import tpot
from tpot import TPOTClassifier
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import dask_ml.model_selection

In [2]:
from dask.distributed import Client, Variable

In [3]:
client=Client() #panggil objek client
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 12,Total memory: 15.85 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:59293,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 15.85 GiB
Comm: tcp://127.0.0.1:59323,Total threads: 3
Dashboard: http://127.0.0.1:59325/status,Memory: 3.96 GiB
Nanny: tcp://127.0.0.1:59298,


In [20]:
df = pd.read_csv('assets/weatherAUS.csv') #membaca data

In [21]:
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [198]:
df.dtypes #memastikan tipe data sudah tepat

Date              object
Location          object
MinTemp          float64
MaxTemp          float64
Rainfall         float64
Evaporation      float64
Sunshine         float64
WindGustDir       object
WindGustSpeed    float64
WindDir9am        object
WindDir3pm        object
WindSpeed9am     float64
WindSpeed3pm     float64
Humidity9am      float64
Humidity3pm      float64
Pressure9am      float64
Pressure3pm      float64
Cloud9am         float64
Cloud3pm         float64
Temp9am          float64
Temp3pm          float64
RainToday         object
RainTomorrow      object
dtype: object

In [22]:
df['Date'] = df.Date.astype('datetime64[ns]') #mengganti tipe data Date

In [200]:
df.shape

(145460, 23)

### Data preparation

#### 1.  Eliminate missing value

In [23]:
df.dropna(inplace=True) 

In [24]:
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
6049,2009-01-01,Cobar,17.9,35.2,0.0,12.0,12.3,SSW,48.0,ENE,...,20.0,13.0,1006.3,1004.4,2.0,5.0,26.6,33.4,No,No
6050,2009-01-02,Cobar,18.4,28.9,0.0,14.8,13.0,S,37.0,SSE,...,30.0,8.0,1012.9,1012.1,1.0,1.0,20.3,27.0,No,No
6052,2009-01-04,Cobar,19.4,37.6,0.0,10.8,10.6,NNE,46.0,NNE,...,42.0,22.0,1012.3,1009.2,1.0,6.0,28.7,34.9,No,No
6053,2009-01-05,Cobar,21.9,38.4,0.0,11.4,12.2,WNW,31.0,WNW,...,37.0,22.0,1012.7,1009.1,1.0,5.0,29.1,35.6,No,No
6054,2009-01-06,Cobar,24.2,41.0,0.0,11.2,8.4,WNW,35.0,NW,...,19.0,15.0,1010.7,1007.4,1.0,6.0,33.6,37.6,No,No


#### 2. Encoding data

In [25]:
un=df.WindGustDir.unique()
un

array(['SSW', 'S', 'NNE', 'WNW', 'N', 'SE', 'ENE', 'NE', 'E', 'SW', 'W',
       'WSW', 'NNW', 'ESE', 'SSE', 'NW'], dtype=object)

In [26]:
#label encoding
column = np.array(['WindGustDir','WindDir9am','WindDir3pm']) 
for col in column:
    df[col]=df[col].map({'SSW':0,'S':1,'NNE':2,'S':3,
                         'WNW':4,'N':5,'SE':6,'ENE':7,
                         'NE':8,'E':9,'SW':10,'W':11,'WSW':12,
                         'NNW':13,'ESE':14,'SSE':15,'NW':16})


In [27]:
column = np.array(['RainToday','RainTomorrow'])
for col in column:
    df[col]=df[col].map({'Yes':1,'No':0})

In [206]:
# column = np.array(['WindGustDir','WindDir9am'])
# for col in column:
#     for i in range(len(df[col])):
#         print(df[col].iloc[i])

In [28]:
df__=df.copy()
df__=df__.reset_index(drop=True)

In [29]:
df__=df__.drop(columns=['Location','Date'])

In [30]:
df__.head(3)

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,17.9,35.2,0.0,12.0,12.3,0,48.0,7,10,6.0,...,20.0,13.0,1006.3,1004.4,2.0,5.0,26.6,33.4,0,0
1,18.4,28.9,0.0,14.8,13.0,3,37.0,15,15,19.0,...,30.0,8.0,1012.9,1012.1,1.0,1.0,20.3,27.0,0,0
2,19.4,37.6,0.0,10.8,10.6,2,46.0,2,13,30.0,...,42.0,22.0,1012.3,1009.2,1.0,6.0,28.7,34.9,0,0


In [260]:
df__.dtypes

MinTemp          float64
MaxTemp          float64
Rainfall         float64
Evaporation      float64
Sunshine         float64
WindGustDir        int64
WindGustSpeed    float64
WindDir9am         int64
WindDir3pm         int64
WindSpeed9am     float64
WindSpeed3pm     float64
Humidity9am      float64
Humidity3pm      float64
Pressure9am      float64
Pressure3pm      float64
Cloud9am         float64
Cloud3pm         float64
Temp9am          float64
Temp3pm          float64
RainToday          int64
RainTomorrow       int64
dtype: object

In [210]:
# X,y= df__.iloc[:,:-1],df__.iloc[:,-1]

In [32]:
digits = load_digits()
train,test=train_test_split(df__,train_size=0.8,test_size=0.2)

In [33]:
X_train = train.iloc[:,:-1]
y_train = train.iloc[:,-1]

In [34]:
X_test = test.iloc[:,:-1]
y_test = test.iloc[:,-1]

In [237]:
# TP = TPOTClassifier(generations=3,population_size=10,cv=5,n_jobs=-1,config_dict=tpot.config.classifier_config_dict_light,use_dask=True)

In [35]:
TP = TPOTClassifier(generations=3,population_size=10,cv=5,n_jobs=-1,use_dask=True)

In [266]:
TP.fit(X_train,y_train)

distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distribu

TPOTClassifier(generations=3, n_jobs=-1, population_size=10, use_dask=True)

In [270]:
TP.export('best_model.py')

In [36]:
from sklearn.ensemble import RandomForestClassifier

In [37]:
exported_pipeline = RandomForestClassifier(bootstrap=False, criterion="gini", max_features=0.55, min_samples_leaf=10, min_samples_split=3, n_estimators=100)

exported_pipeline.fit(X_train, y_train)
results = exported_pipeline.predict(X_test)

In [42]:
exported_pipeline.score(X_test,y_test)

0.859181141439206

In [4]:
client.close()